In [1]:
from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda

import requests
import pandas as pd
import numpy as np
import ast
pd.options.display.max_columns=None

In [2]:
attack=pd.read_csv('../ETL/datos/attacks_limpieza_completa.csv', index_col=0)
#df=pd.read_csv('../ETL/datos/df_API_paises.csv', index_col=0)

## UNA CLASE 

In [15]:
class concat_clean_merge():
     
    def paises(producto):

        df_concat=pd.DataFrame()
        dict={'USA':[39.783730,-100.445882],'Australia':[-24.77610,134.755], 'South Africa':[-28.8166236,24.991639],\
            'New Zealand':[-41.5000831,172.8344077],'Papua New Guinea':[-5.6816069,144.2489081] }
            
        for k,v in dict.items():
                
            url = f'http://www.7timer.info/bin/api.pl?lon=-{v[1]}&lat={v[0]}&product={producto}&output=json'

            response = requests.get(url)
            df=pd.json_normalize(response.json()["dataseries"])
            df["latitud"] = v[0]
            df["longitud"] = v[1]
            df['paises']= k
            df_concat=pd.concat([df_concat,df], axis=0, ignore_index=True)

        return df_concat

    def limpiar_attack (df):

        df_attack=df[(df["country"]=="usa") | (df["country"]=="australia")|\
            (df["country"]=="south africa") | (df["country"]=="papua new guinea")|(df["country"]=="new zealand")]

        df_attack.rename(columns={'country':'id_paises'}, inplace=True)

        df_final=df_attack.groupby('id_paises')['year','age','latitud','longitud','fatal_N'].agg(['mean'])
        df_final=df_final.reset_index()

        return df_final
    
    def limpieza_clima(df):

        #Limpiamos la columna de rh_profile primero a series y luego a columnas en el bbucle for
        #df['rh_profile']=df['rh_profile'].apply(ast.literal_eval)
        rh = df['rh_profile'].apply(pd.Series)

        for i in range(len(rh.columns)):
            rh_col0=rh[i].apply(pd.Series)
            nombre="rh"+ rh_col0["layer"][115]
            valores= list(rh_col0["rh"])
            df.insert(i, nombre, valores)
            
        #Eliminamos la columna que ya no necesitamos
        df=df.drop(['rh_profile'], axis=1)
        
        #Hacemos lo mismo para la columnas wind_profile
        #df['wind_profile']=df['wind_profile'].apply(ast.literal_eval)
        wind = df['wind_profile'].apply(pd.Series)

        for i in range(len(wind.columns)):
            wind_col0=wind[i].apply(pd.Series)
            nombre="wind_"+ wind_col0["layer"][1]
            valores= list(wind_col0["direction"])
            df.insert(i, nombre, valores)
            
        df=df.drop(['wind_profile'], axis=1)
        
        df2=df.groupby('paises').agg(['mean'])
        df2=df2.reset_index()
        df2.rename(columns={'paises':'id_paises'}, inplace=True)
        def minuscula(col):
            try:
                col=col.lower()
                return col
            except:
                pass
        df2["id_paises"]=df2["id_paises"].apply(minuscula)

        return df2

    def merge(df_left, df_right): #Esta función está persanda para hacer un left. Para que se tenga en cuenta

        merge_final=pd.merge(left=df_left, right=df_right, how='left', left_on='id_paises', right_on='id_paises')
        merge_final.to_csv('merge_clima_ataques.csv')
        
        return merge_final

In [10]:
clase=concat_clean_merge

In [11]:
df1=concat_clean_merge.paises('meteo')
df1.head()

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud,paises
0,3,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 13}, {'layer': '900m...","[{'layer': '950mb', 'direction': 65, 'speed': ...",23,2,10,1016,rain,2,0,90,3,39.78373,-100.445882,USA
1,6,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",24,2,9,1016,rain,2,0,90,2,39.78373,-100.445882,USA
2,9,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 85, 'speed': ...",24,2,8,1014,rain,2,0,110,2,39.78373,-100.445882,USA
3,12,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 120, 'speed':...",24,2,9,1014,none,2,0,140,2,39.78373,-100.445882,USA
4,15,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 170, 'speed':...",24,2,9,1015,none,2,0,165,2,39.78373,-100.445882,USA


In [12]:
df_attack=concat_clean_merge.limpiar_attack(attack)
df_attack.head()

/tmp/ipykernel_33074/3105811130.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_attack.rename(columns={'country':'id_paises'}, inplace=True)
/tmp/ipykernel_33074/3105811130.py:29: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_final=df_attack.groupby('id_paises')['year','age','latitud','longitud','fatal_N'].agg(['mean'])


,id_paises,year,age,latitud,longitud,fatal_N
,,mean,mean,mean,mean,mean
0,australia,1945.205479,27.527397,-24.776109,134.755000,0.690259
1,new zealand,1941.134921,27.174603,-41.500083,172.834408,0.706349
2,papua new guinea,1914.460317,24.666667,-5.681607,144.248908,0.531746
3,south africa,1960.784965,26.159091,-28.816624,24.991639,0.709790
4,usa,1971.065544,26.842057,39.783730,-100.445882,0.821575


In [16]:
df_clima=concat_clean_merge.limpieza_clima(df1)
df_clima.head()

/tmp/ipykernel_33074/1540885676.py:61: FutureWarning: ['prec_type'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  df2=df.groupby('paises').agg(['mean'])


,id_paises,wind_950mb,wind_900mb,wind_850mb,wind_800mb,wind_750mb,wind_700mb,wind_650mb,wind_600mb,wind_550mb,wind_500mb,wind_450mb,wind_400mb,wind_350mb,wind_300mb,wind_250mb,wind_200mb,rh950mb,rh900mb,rh850mb,rh800mb,rh750mb,rh700mb,rh650mb,rh600mb,rh550mb,rh500mb,rh450mb,rh400mb,rh350mb,rh300mb,rh250mb,rh200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud,longitud
,,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
0,australia,103.125000,102.890625,124.531250,167.734375,166.250000,203.281250,264.531250,257.187500,261.250000,260.078125,258.359375,256.406250,285.000000,278.593750,282.187500,280.312500,12.687500,12.156250,9.625000,5.406250,2.656250,1.687500,1.187500,0.687500,0.625000,0.984375,0.203125,-0.25000,0.187500,2.796875,5.515625,2.328125,97.5,5.531250,-9999.0,-9999.0,-9999.0,21.718750,2.406250,10.265625,1022.328125,2.593750,0.0,1.642202e+151,3.09375,-24.776100,134.755000
1,new zealand,215.234375,222.187500,215.390625,207.343750,227.812500,252.500000,254.609375,268.046875,270.468750,265.625000,278.984375,274.843750,275.859375,273.125000,275.703125,289.375000,11.937500,12.953125,10.015625,6.640625,5.125000,3.203125,4.859375,6.031250,6.296875,6.906250,7.984375,9.96875,11.640625,11.921875,10.531250,-0.578125,97.5,8.843750,-9999.0,-9999.0,-9999.0,11.500000,8.046875,11.734375,1024.046875,2.515625,0.0,3.284456e+167,3.68750,-41.500083,172.834408
2,papua new guinea,101.406250,101.328125,104.062500,105.078125,84.687500,66.953125,107.812500,134.375000,116.875000,85.234375,119.296875,201.562500,199.140625,266.875000,277.421875,281.171875,13.734375,11.671875,7.171875,2.796875,0.437500,-0.828125,-3.109375,-3.937500,-4.000000,-4.000000,-3.437500,-2.84375,-2.046875,-1.968750,-1.359375,0.343750,97.5,2.562500,-9999.0,-9999.0,-9999.0,25.765625,1.625000,11.281250,1010.984375,2.531250,0.0,1.720470e+164,3.68750,-5.681607,144.248908
3,south africa,191.953125,221.562500,247.031250,288.750000,300.312500,289.765625,273.359375,264.453125,261.640625,269.218750,270.468750,278.750000,273.750000,271.796875,271.640625,271.796875,12.546875,11.812500,10.921875,9.015625,8.171875,7.406250,6.187500,5.031250,4.390625,4.562500,6.109375,7.18750,7.546875,6.890625,7.859375,6.937500,97.5,6.875000,-9999.0,-9999.0,-9999.0,17.859375,2.875000,12.406250,1018.109375,5.281250,0.0,5.394555e+157,3.25000,-28.816624,24.991639
4,usa,155.859375,160.234375,194.140625,223.125000,248.828125,251.328125,248.906250,231.015625,217.500000,215.546875,225.312500,230.234375,254.687500,252.031250,249.921875,257.656250,10.656250,11.953125,11.000000,9.546875,8.734375,8.640625,7.906250,7.359375,6.093750,5.812500,6.531250,7.56250,9.250000,9.640625,8.890625,1.781250,97.5,6.046875,-9999.0,-9999.0,-9999.0,23.421875,-0.859375,8.406250,1014.203125,3.671875,0.0,1.420330e+168,2.31250,39.783730,-100.445882


In [17]:
merge_final=concat_clean_merge.merge(df_clima,df_attack)
merge_final

/tmp/ipykernel_33074/1540885676.py:76: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  merge_final=pd.merge(left=df_left, right=df_right, how='left', left_on='id_paises', right_on='id_paises')


,id_paises,wind_950mb,wind_900mb,wind_850mb,wind_800mb,wind_750mb,wind_700mb,wind_650mb,wind_600mb,wind_550mb,wind_500mb,wind_450mb,wind_400mb,wind_350mb,wind_300mb,wind_250mb,wind_200mb,rh950mb,rh900mb,rh850mb,rh800mb,rh750mb,rh700mb,rh650mb,rh600mb,rh550mb,rh500mb,rh450mb,rh400mb,rh350mb,rh300mb,rh250mb,rh200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,latitud_x,longitud_x,year,age,latitud_y,longitud_y,fatal_N
,,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
0,australia,103.125000,102.890625,124.531250,167.734375,166.250000,203.281250,264.531250,257.187500,261.250000,260.078125,258.359375,256.406250,285.000000,278.593750,282.187500,280.312500,12.687500,12.156250,9.625000,5.406250,2.656250,1.687500,1.187500,0.687500,0.625000,0.984375,0.203125,-0.25000,0.187500,2.796875,5.515625,2.328125,97.5,5.531250,-9999.0,-9999.0,-9999.0,21.718750,2.406250,10.265625,1022.328125,2.593750,0.0,1.642202e+151,3.09375,-24.776100,134.755000,1945.205479,27.527397,-24.776109,134.755000,0.690259
1,new zealand,215.234375,222.187500,215.390625,207.343750,227.812500,252.500000,254.609375,268.046875,270.468750,265.625000,278.984375,274.843750,275.859375,273.125000,275.703125,289.375000,11.937500,12.953125,10.015625,6.640625,5.125000,3.203125,4.859375,6.031250,6.296875,6.906250,7.984375,9.96875,11.640625,11.921875,10.531250,-0.578125,97.5,8.843750,-9999.0,-9999.0,-9999.0,11.500000,8.046875,11.734375,1024.046875,2.515625,0.0,3.284456e+167,3.68750,-41.500083,172.834408,1941.134921,27.174603,-41.500083,172.834408,0.706349
2,papua new guinea,101.406250,101.328125,104.062500,105.078125,84.687500,66.953125,107.812500,134.375000,116.875000,85.234375,119.296875,201.562500,199.140625,266.875000,277.421875,281.171875,13.734375,11.671875,7.171875,2.796875,0.437500,-0.828125,-3.109375,-3.937500,-4.000000,-4.000000,-3.437500,-2.84375,-2.046875,-1.968750,-1.359375,0.343750,97.5,2.562500,-9999.0,-9999.0,-9999.0,25.765625,1.625000,11.281250,1010.984375,2.531250,0.0,1.720470e+164,3.68750,-5.681607,144.248908,1914.460317,24.666667,-5.681607,144.248908,0.531746
3,south africa,191.953125,221.562500,247.031250,288.750000,300.312500,289.765625,273.359375,264.453125,261.640625,269.218750,270.468750,278.750000,273.750000,271.796875,271.640625,271.796875,12.546875,11.812500,10.921875,9.015625,8.171875,7.406250,6.187500,5.031250,4.390625,4.562500,6.109375,7.18750,7.546875,6.890625,7.859375,6.937500,97.5,6.875000,-9999.0,-9999.0,-9999.0,17.859375,2.875000,12.406250,1018.109375,5.281250,0.0,5.394555e+157,3.25000,-28.816624,24.991639,1960.784965,26.159091,-28.816624,24.991639,0.709790
4,usa,155.859375,160.234375,194.140625,223.125000,248.828125,251.328125,248.906250,231.015625,217.500000,215.546875,225.312500,230.234375,254.687500,252.031250,249.921875,257.656250,10.656250,11.953125,11.000000,9.546875,8.734375,8.640625,7.906250,7.359375,6.093750,5.812500,6.531250,7.56250,9.250000,9.640625,8.890625,1.781250,97.5,6.046875,-9999.0,-9999.0,-9999.0,23.421875,-0.859375,8.406250,1014.203125,3.671875,0.0,1.420330e+168,2.31250,39.783730,-100.445882,1971.065544,26.842057,39.783730,-100.445882,0.821575
